Action2	"汽车投诉信息采集：
数据源：http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml
投诉编号，投诉品牌，投诉车系，投诉车型，问题简述，典型问题，投诉时间，投诉状态
可以采用Python爬虫，或者第三方可视化工具"			
1、完成代码（20points）
2、结果正确（20points）

In [ ]:
# 使用request + BeautifulSoup提取12365auto投诉信息
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os    #获取当前工作路径

In [ ]:
#请求URL
#request_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml'
# 根据request_url得到soup
def get_page_content(request_url):
    # 得到页面的内容
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    html=requests.get(request_url,headers=headers,timeout=10)
    content = html.text
     # 通过content创建BeautifulSoup对象
    soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')
    return soup

In [ ]:
#通过BS对象来提取当前页面的内容（方法一）
def analysis(soup):
    #找出投诉框
    temp=soup.find('div',class_='tslb_b')
    #投诉编号	投诉品牌	投诉车系	投诉车型	问题简述	典型问题	投诉时间	投诉状态
    df=pd.DataFrame(columns=['id_','brand','car_model','type_','desc','problem','datetime','status'])#定义一个空的df
    tr_list=temp.find_all('tr')
    for tr in tr_list:
        td_list=tr.find_all('td')
        df_tr=pd.DataFrame()
        if len(td_list)>0:
            df_tr = df_tr.assign(
                id_=[td_list[0].text],
                brand=[td_list[1].text],
                car_model=[td_list[2].text],
                type_=[td_list[3].text],
                desc=[td_list[4].text],
                problem=[td_list[5].text],
                datetime=[td_list[6].text],
                status =[td_list[7].text]
            )
            df=pd.concat([df,df_tr],axis=0,ignore_index=True)
    return df 

request_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml'
bs = get_page_content(request_url)
df =analysis(bs) 
print(df)

In [14]:
#通过BS对象来提取当前页面的内容（方法二）
def analysis(soup):
    #找出投诉框
    temp=soup.find('div',class_='tslb_b')
    #投诉编号	投诉品牌	投诉车系	投诉车型	问题简述	典型问题	投诉时间	投诉状态
    df=pd.DataFrame(columns=['id','brand','car_model','type','desc','problem','datetime','status'])#定义一个空的df
    tr_list=temp.find_all('tr')
    for tr in tr_list:
        temp={}
        td_list=tr.find_all('td')
        #第一个TR没有td,其余八个都有
        #for td in td_list:
        if len(td_list) > 0:
            id,brand,car_model,type,desc,problem,datetime,status = \
            td_list[0].text,td_list[1].text,td_list[2].text,td_list[3].text,td_list[4].text, \
            td_list[5].text,td_list[6].text,td_list[7].text
                
            temp['id'],temp['brand'],temp['car_model'],temp['type'],temp['desc'],temp['problem'], \
            temp['datetime'],temp['status'] = \
            id,brand,car_model,type,desc,problem,datetime,status
                
            df = df.append(temp, ignore_index=True)
    return df 

request_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml'
bs = get_page_content(request_url)
df =analysis(bs) 
print(df)

        id    brand   car_model                           type  \
0   515601     东风启辰       启辰T60    2018款 1.6L CVT 智悦版(ISS) 国VI   
1   515594    长安马自达   马自达3 昂克赛拉              2020款 1.5L 自动 质美版   
2   515592    一汽-大众          探岳          2020款 330TSI 两驱 豪华智联版   
3   515589  一汽-大众奥迪       奥迪A4L          2018款 2.0T 时尚型 30周年年型   
4   515577     吉利汽车          缤瑞              2018款 14T CVT 缤纷版   
5   515559    一汽-大众         高尔夫             2014款 1.6L  手动 时尚型   
6   515543     北京现代          名图          2017款 1.6T 自动 智能型 GLS   
7   515538     众泰汽车        云100                 2014款 云100 舒适型   
8   515537     东风日产          逍客             2016款 2.0L CVT 精英版   
9   515535     上汽大众          朗逸           2013款 1.6L 自动 舒适版 改款   
10  515534     广汽丰田         凯美瑞              2018款 2.5G 自动 豪华版   
11  515533   东风悦达起亚        起亚K2           2017款 1.4L 自动 GLS 三厢   
12  515532     一汽丰田      RAV4荣放           2015款 2.0L CVT 两驱都市版   
13  515525    一汽-大众    C-TREK蔚领              2018款 1.5L 自动 舒适型   
14  515523

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:179: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [ ]:
page_num = 20
base_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-'

result = pd.DataFrame(columns = ['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'datetime', 'status'])
for i in range(page_num):
    request_url = base_url+str(i+1)+'.shtml'
    soup = get_page_content(request_url)
    print(request_url)
    df = analysis(soup)
    print(df)
    result = result.append(df)
result.to_csv(r'C:\Users\Administrator\BI\calss1-2\car_complain.csv', index=False,encoding="utf_8_sig") #数据写入，index=False表示不加索引